In [33]:
import sympy as sp

# Define the symbols
alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff, S_x, S_y, S_z = sp.symbols(
    'alpha beta gama X Y Z L v1 v2 v3 n1 n2 n3 ff S_x S_y S_z', real=True)
# Define the symbols for S
S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15 = sp.symbols("S1:16", real=True)
# observations for minsquare
f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10 = sp.symbols('f_obs1:11', real=True)

# Sip and TCP
Sip = sp.Matrix([S_x, S_y, S_z])
TCP = sp.Matrix([X, Y, Z])

# Rotation matrices Rx, Ry, Rz
Rx = sp.Matrix([[1, 0, 0], [0, sp.cos(alpha), -sp.sin(alpha)], [0, sp.sin(alpha), sp.cos(alpha)]])
Ry = sp.Matrix([[sp.cos(beta), 0, sp.sin(beta)], [0, 1, 0], [-sp.sin(beta), 0, sp.cos(beta)]])
Rz = sp.Matrix([[sp.cos(gama), -sp.sin(gama), 0], [sp.sin(gama), sp.cos(gama), 0], [0, 0, 1]])
R = sp.simplify(Rx * Ry * Rz)

In [36]:
Rx @ Ry

Matrix([
[            cos(beta),          0,             sin(beta)],
[ sin(alpha)*sin(beta), cos(alpha), -sin(alpha)*cos(beta)],
[-sin(beta)*cos(alpha), sin(alpha),  cos(alpha)*cos(beta)]])

In [2]:
import sympy as sp

# Define the symbols
alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff, S_x, S_y, S_z = sp.symbols(
    'alpha beta gama X Y Z L v1 v2 v3 n1 n2 n3 ff S_x S_y S_z', real=True)
# Define the symbols for S
S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15 = sp.symbols("S1:16", real=True)
# observations for minsquare
f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10 = sp.symbols('f_obs1:11', real=True)

# Sip and TCP
Sip = sp.Matrix([S_x, S_y, S_z])
TCP = sp.Matrix([X, Y, Z])

# Rotation matrices Rx, Ry, Rz
Rx = sp.Matrix([[1, 0, 0], [0, sp.cos(alpha), -sp.sin(alpha)], [0, sp.sin(alpha), sp.cos(alpha)]])
Ry = sp.Matrix([[sp.cos(beta), 0, sp.sin(beta)], [0, 1, 0], [-sp.sin(beta), 0, sp.cos(beta)]])
Rz = sp.Matrix([[sp.cos(gama), -sp.sin(gama), 0], [sp.sin(gama), sp.cos(gama), 0], [0, 0, 1]])
R = sp.simplify(Rx * Ry * Rz)

# Vector v and calculations for x and o
v = sp.Matrix([v1, v2, v3])
x = sp.Matrix([X, Y, Z]) - L * R * v
o = R * v

# Calculate 'a' using dot products
a = sp.simplify(sp.sqrt(((-R.row(0).dot(v) * Z) / (R.row(2).dot(v)))**2 +
                        (-(R.row(1).dot(v) * Z) / (R.row(2).dot(v)))**2 + Z * Z))

# Calculate Q
Q = a * o + TCP

# Project Sip onto the line defined by o and Q
Sip_proj = Sip + (o.dot(Q) - o.dot(Sip)) * (x - Sip) / (o.dot(x) - o.dot(Sip))

# s_2 vector and s_1 as the cross product of o and s_2
s_2 = R @ sp.Matrix([n1, n2, n3])
s_1 = o.cross(s_2)

# Calculate k_1 and k_2
k_1 = (Sip_proj - Q).dot(s_1) / s_1.norm()**2
k_2 = (Sip_proj - Q).dot(s_2) / s_2.norm()**2

# Q_t as a zero vector (for 2D image plane coordinates)
Q_t = sp.Matrix([0, 0])

# Calculate image plane coordinates f_1 and f_2
f_1 = Q_t[0] + k_1 * ff / (a + L - ff)
f_2 = Q_t[1] + k_2 * ff / (a + L - ff)


M = sp.Matrix(5, 3, (S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15))

# Define the matrix to store the results of f1 and f2 for all S
results = sp.Matrix(10, 1, [0] * 10)

for i in range(5):
    # Substitute the symbols from M for S_x, S_y, S_z in f_1 and f_2
    f1_i = f_1.subs({S_x: M[i, 0], S_y: M[i, 1], S_z: M[i, 2]})
    f2_i = f_2.subs({S_x: M[i, 0], S_y: M[i, 1], S_z: M[i, 2]})
    # Store the results in the matrix (also convert to mm)
    results[2 * i, 0] = f1_i * 1e3
    results[2 * i + 1, 0] = f2_i * 1e3


f = sp.Matrix(10, 1, (f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10))

# MinSquare functional
diff = results - f
F_obs = (diff[0] * diff[0] + diff[1] * diff[1] +
         diff[2] * diff[2] + diff[3] * diff[3] +
         diff[4] * diff[4] + diff[5] * diff[5] +
         diff[6] * diff[6] + diff[7] * diff[7] +
         diff[8] * diff[8] + diff[9] * diff[9])

In [3]:
R

Matrix([
[                                  cos(beta)*cos(gama),                                   -sin(gama)*cos(beta),             sin(beta)],
[sin(alpha)*sin(beta)*cos(gama) + sin(gama)*cos(alpha), -sin(alpha)*sin(beta)*sin(gama) + cos(alpha)*cos(gama), -sin(alpha)*cos(beta)],
[sin(alpha)*sin(gama) - sin(beta)*cos(alpha)*cos(gama),  sin(alpha)*cos(gama) + sin(beta)*sin(gama)*cos(alpha),  cos(alpha)*cos(beta)]])

In [4]:
import inspect

args = [X, Y, Z, alpha, beta, gama, L, v1, v2, v3, n1, n2, n3, ff,
        S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15,
        f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10]
F_obs_lambd = sp.lambdify(args, F_obs, modules='jax', cse=True)

function_text = inspect.getsource(F_obs_lambd)


def valueF_jax(x, v, n, L, ff, S, f):
    return F_obs_lambd(x[0], x[1], x[2], x[3], x[4], x[5], L, v[0], v[1], v[2], n[0], n[1], n[2], ff,
                       S[0], S[1], S[2], S[3], S[4], S[5], S[6], S[7], S[8], S[9], S[10], S[11], S[12], S[13], S[14],
                       f[0], f[1], f[2], f[3], f[4], f[5], f[6], f[7], f[8], f[9])

In [6]:
print(function_text)


def _lambdifygenerated(X, Y, Z, alpha, beta, gama, L, v1, v2, v3, n1, n2, n3, ff, S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15, f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10):
    x0 = cos(alpha)
    x1 = cos(beta)
    x2 = x0*x1
    x3 = v3*x2
    x4 = sin(alpha)
    x5 = cos(gama)
    x6 = x4*x5
    x7 = sin(beta)
    x8 = sin(gama)
    x9 = x0*x8
    x10 = x6 + x7*x9
    x11 = v2*x10
    x12 = x4*x8
    x13 = x0*x5
    x14 = x12 - x13*x7
    x15 = v1*x14
    x16 = x11 + x15 + x3
    x17 = sqrt(v1**2 + v2**2 + v3**2)*abs(Z/x16)
    x18 = (L - ff + x17)**(-1.0)
    x19 = x1*x5
    x20 = x1*x8
    x21 = n1*x19 - n2*x20 + n3*x7
    x22 = n1*x14 + n2*x10 + n3*x2
    x23 = x1*x4
    x24 = n3*x23
    x25 = x6*x7 + x9
    x26 = n1*x25
    x27 = x12*x7 - x13
    x28 = n2*x27 + x24 - x26
    x29 = (x21**2 + x22**2 + x28**2)**(-1.0)
    x30 = -X
    x31 = v3*x7
    x32 = v1*x19
    x33 = -L*v2*x1*x8 + L*x31 + L*x32 + x30
    x34 = v3*x23
  

In [ ]:
import jax.numpy as jnp
from jax.numpy import sin, cos, sqrt, abs
import jax

# Set JAX to use 64-bit floats
jax.config.update('jax_enable_x64', True)

# derivative of abs_val using jax
gradF_jax = jax.grad(valueF_jax, argnums=0)
hessF_jax = jax.hessian(valueF_jax, argnums=0)

valueF_jax_compiled = jax.jit(valueF_jax)
grafF_compiled = jax.jit(gradF_jax)
hessF_compiled = jax.jit(hessF_jax)

In [ ]:
import pandas as pd
import numpy as np

# Reading data from Excel
images = pd.read_excel('body.xlsx', usecols='A:J', header=None).to_numpy()
camera_coordinates = pd.read_excel('souradnice.xlsx', usecols='A:F', header=None).to_numpy()
X = camera_coordinates

# Converting degrees to radians
X[:, 3:5] = X[:, 3:5] * np.pi / 180

# Extracting columns from images
aa, bb, cc, dd, ee = images[:, 0:2], images[:, 2:4], images[:, 4:6], images[:, 6:8], images[:, 8:10]

# Transformation matrices
Transform_matrix = np.array([[0, -1, 0], [1, 0, 0], [0, 0, -1]])
Transform_matrix_rotation = np.array([[0, 1, 0], [-1, 0, 0], [0, 0, 1]])

S = np.array([0, 0, 0, 0.05, 0.05, 0, 0.05, -0.05, 0, -0.05, -0.05, 0, -0.05, 0.05, 0]) * 1e3

v = np.array([0, 0, -1])
n = np.array([0, 1, 0])

L = 12.5 * 1e-3
L = 9.5 * 1e-3

F_val = []
for j in range(1):  # In Python, loop starts from 0
    x = X[j, :].T
    x = np.concatenate((Transform_matrix @ x[:3], Transform_matrix_rotation @ x[3:6]))

    L = min([309.5 * 1e-3, 309.5 * 1e-3 + x[2] * 1e-3])
    x[2] = max([x[2], 0])

    SS = np.array([2592 / 2, 1944 / 2])

    A, B, C, D, E = aa[j, :].T - SS, bb[j, :].T - SS, cc[j, :].T - SS, dd[j, :].T - SS, ee[j, :].T - SS

    f_presc = np.array([A, B, C, D, E]).ravel() * 2.2 / 1e6 * 1e3
    f_presc[[2, 3, 6, 7]] = f_presc[[6, 7, 2, 3]]

    x01 = x + np.array([1, 1, 1, 0.00001, 0.00001, 0.00001])

    # Optimization function (fmin_beres) should be defined or replaced with an equivalent
    # [x, it] = fmin_beres(S * 1e-3, f_presc, v, n, L, 1000, 1e-6)
    # x[:3] = x[:3] * 1e3

    # Function value_f and valueF should be defined
    # f = value_f(x[0] * 1e-3, x[1] * 1e-3, x[2] * 1e-3, x[3], x[4], x[5], S * 1e-3, v, n, L)
    # F_val.append(valueF(x[0] * 1e-3, x[1] * 1e-3, x[2] * 1e-3, x[3], x[4], x[5], S * 1e-3, f_presc, v, n, L))

In [ ]:
x01[:3] = x01[:3]*1e-3

In [ ]:
valueF_jax_compiled(jnp.array(x01,dtype=jnp.float64), jnp.array(v,dtype=jnp.float64), 
                    jnp.array(n, dtype=jnp.float64), L, 0.008, 
                    jnp.array(S, dtype=jnp.float64)*1e-3, jnp.array(f_presc, dtype=jnp.float64))

In [ ]:
grafF_compiled(jnp.array(x01, dtype=jnp.float64), jnp.array(v, dtype=jnp.float64),
               jnp.array(n, dtype=jnp.float64), L, 0.008,
               jnp.array(S, dtype=jnp.float64)*1e-3, jnp.array(f_presc, dtype=jnp.float64))

In [ ]:
hessF_compiled(jnp.array(x01, dtype=jnp.float64), jnp.array(v, dtype=jnp.float64),
               jnp.array(n, dtype=jnp.float64), L, 0.008,
               jnp.array(S, dtype=jnp.float64)*1e-3, jnp.array(f_presc, dtype=jnp.float64))